In [6]:
!pip install langchain
!pip install langchain-community
!pip install pypdf
!pip install sentence-transformers
!pip install faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 14.5 MB/s eta 0:00:0000:0100:01


In [1]:
import os
import json
import boto3
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.vectorstores import FAISS, Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, GenerationConfig, AutoModelForCausalLM
from langchain_community.llms import HuggingFacePipeline
from langchain_community.llms import SagemakerEndpoint
from langchain_community.llms.sagemaker_endpoint import LLMContentHandler
from typing import Dict
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA


import warnings
warnings.filterwarnings('ignore')

In [70]:
# file_name1 = "uber_2021.pdf"
# file_name2 = "lyft_2021.pdf"


# loaders = [
#     PyPDFLoader("docs/uber_2021.pdf"),
#     PyPDFLoader("docs/lyft_2021.pdf")
# ]




file_name1 = "MIZUHO_FINANCIAL_GROUP_1.pdf"
file_name2 = "MIZUHO_FINANCIAL_GROUP_2.pdf"

loaders = [
    PyPDFLoader("docs/MIZUHO_FINANCIAL_GROUP_1.pdf"),
    PyPDFLoader("docs/MIZUHO_FINANCIAL_GROUP_2.pdf")
]

docs = []
for loader in loaders:
    docs.extend(loader.load())
    

text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1048,
    chunk_overlap = 256,
    length_function = len
)

splits = text_splitter.split_documents(docs)
    
print(len(docs))

51


In [71]:
class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
        input_str = json.dumps({"inputs": prompt, "parameters": model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json[0]["generated_text"]

content_handler = ContentHandler()

embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vector_db = FAISS.from_documents(documents=splits, embedding=embedding)
# vector_db = Chroma.from_documents(documents=splits, embedding=embedding)

runtime = boto3.client("runtime.sagemaker")

llm=SagemakerEndpoint(
        endpoint_name="mistral-llm",
        client=runtime,
        model_kwargs={"temperature": 0.8},
        content_handler=content_handler
    )
print("Done")

Done


In [72]:

template = """
Answer the question as detailed as possible from the provided context, make sure to provide all the details.
If the answer is not in provided context just say, "Answer is not available in the context.", please don't provide the wrong answer.

Provide complete answer. Provide Confidence Score as well.

Context:\n {context}?\n

Question: \n Get information on {question} \n

Confidence Score:"""

prompt = PromptTemplate(template = template, input_variables = ["context", "question"])

chain = RetrievalQA.from_chain_type(
    llm = llm,
    retriever = vector_db.as_retriever(search_kwargs={"k":12}),
    return_source_documents=False,
    chain_type = "stuff",
    chain_type_kwargs={"prompt":prompt}
)
print("Done")

Done


#### Compare - on parameters

In [73]:

question = "Risk Factors" + " from "+file_name1
file1_response = chain({"query":question})

question = "Risk Factors" + " from "+file_name2
file2_response = chain({"query":question})

print(file1_response)

{'query': 'Risk Factors from MIZUHO_FINANCIAL_GROUP_1.pdf', 'result': '\nAnswer the question as detailed as possible from the provided context, make sure to provide all the details.\nIf the answer is not in provided context just say, "Answer is not available in the context.", please don\'t provide the wrong answer.\n\nProvide complete answer. Provide Confidence Score as well.\n\nContext:\n EXHIBIT  A\n \nAMERICAN DEPOSIT ARY SHARES\n(Each American Depositary Share\nrepresents 1/500th of one deposited Share)\nTHE BANK OF NEW  YORK\nAMERICAN DEPOSIT ARY RECEIPT\nFOR SHARES OF COMMON ST OCK OF\nMIZUHO FINANCIAL  GROUP , INC.\n(INCORPORA TED UNDER THE LA WS OF JAP AN)\nThe Bank of New York, as depositary (hereinafter called the “Depositary”), hereby certifies that                                                          \n                                                                         , or registered assigns IS THE OWNER OF                                                          

In [74]:
def convert_to_json_reponse(str_input):
    internal_json = '''
        "Question":{},
        "Confidence Score":{},
        "Answer":{}
    '''
    # slicing question
    sub1 = "Question:"
    sub2 = "Confidence Score:"
 
    # getting elements in between using split() and join()
    question = ''.join(str_input.split(sub1)[1].split(sub2)[0]).strip()
    question = "\""+question+"\""
    
    # slicing Confidence Score
    sub1 = "Confidence Score:"
    sub2 = "Answer:"
 
    # getting elements in between using split() and join()
    conf_score = ''.join(str_input.split(sub1)[1].split(sub2)[0]).strip()
    conf_score = "\""+conf_score+"\""
    
    # slicing answer
    sub1 = "Answer:"
 
    # getting elements in between using split() and join()
    answer = ''.join(str_input.split(sub1)[1]).strip()
    answer = "\""+answer+"\""
    
    result = internal_json.format(question, conf_score, answer)
    
    result = "{"+result+"}"
    
    return result

def slice_and_convert_to_json(file1_response, file2_response):
    file1_response_result = file1_response.get("result")
    file2_response_result = file2_response.get("result")

    sub_str = "Question:"
    
    file1_response_output = "Question:\n"
    file2_response_output = "Question:\n"

    file1_response_output = file1_response_output + file1_response_result.rsplit(sub_str, 1)[1]
    # print(response_file1_output)

    # print("\n\n-------------------------")
    file2_response_output = file2_response_output + file2_response_result.rsplit(sub_str, 1)[1]
    # print(response_file2_output)
    
    json_response = '''
        "{}":{},
        "{}":{}
    '''

    result = json_response.format(file_name1, convert_to_json_reponse(file1_response_output), file_name2, convert_to_json_reponse(file2_response_output))
    result = "{"+result+"}"
    
    json_result = json.loads(result)

    return json_result

In [75]:
print(slice_and_convert_to_json(file1_response, file2_response))

{'MIZUHO_FINANCIAL_GROUP_1.pdf': {'Question': 'Get information on Risk Factors from MIZUHO_FINANCIAL_GROUP_1.pdf', 'Confidence Score': '100%', 'Answer': 'The context of the provided document does not include specific information on risk factors related to Mizuho Financial Group, Inc. However, there are provisions in the Deposit Agreement that address compliance with U.S. securities laws and submission to jurisdiction in New York. Violations of these provisions could potentially lead to risks for the investors. Additionally, the Company Law of Japan is referenced throughout the agreement, and any changes to that'}, 'MIZUHO_FINANCIAL_GROUP_2.pdf': {'Question': 'Get information on Risk Factors from MIZUHO_FINANCIAL_GROUP_2.pdf', 'Confidence Score': '100%', 'Answer': 'The context does not directly provide information on risk factors related to Mizuho Financial Group, Inc. from the given MIZUHO_FINANCIAL_GROUP_2.pdf document. However, the context does mention a section (Section 22) in the D